In [13]:
import pandas as pd
import itertools

df = pd.read_csv("NAI_story_data/NAI_story_data.csv")

In [72]:
# Put starting prompt and result together
df["full_story"] = df["prompt"] + df["result"]

# Start building blocks for qualtrics survey

# Create IDs for different prompt combinations
genre_li = df["prompt_label"].unique()
preset_li = df["preset_label"].unique()

genre_preset_li = list(itertools.product(preset_li, genre_li))

# create ID prefixes by using the first 3 letters of preset and identifer for genre
# when adding more presets and/or genres make sure IDs stay unique
story_id_dict = {}

for comb in genre_preset_li:
    id_str = comb[0][:3] + "_"
    genre = comb[1]
    if genre == "High Fantasy":
        id_str += "HF"
    elif genre == "Horror":
        id_str += "HOR"
    elif genre == "Hard Sci-fi":
        id_str += "HSF"
    elif genre == "Historical Romance":
        id_str += "HR"
        
    id_str =  id_str.upper()
    story_id_dict[id_str] = 0

# Create string to later write into a file for Qualtrics' advanced txt format
qualtrics_str = "[[AdvancedFormat]]\n\n[[Block:Stories]]\n"

for index, row in df.iterrows():
    
    # determine story id
    story_id_prefix = row["preset_label"][:3] + "_"
    genre = row["prompt_label"]
    
    if genre == "High Fantasy":
        story_id_prefix += "HF"
    elif genre == "Horror":
        story_id_prefix += "HOR"
    elif genre == "Hard Sci-fi":
        story_id_prefix += "HSF"
    elif genre == "Historical Romance":
        story_id_prefix += "HR"
    
    story_id_prefix = story_id_prefix.upper()
    
    story_id = story_id_prefix + "_" + str(story_id_dict[story_id_prefix]+1)
    
    # increase counter for id
    story_id_dict[story_id_prefix] += 1
    
    # Write to qualtrics string
    qualtrics_str += "\n[[Text]]"
    qualtrics_str += "\n[[ID:" + story_id + "]]\n"
    qualtrics_str += row["full_story"]
    qualtrics_str += "\n"

In [76]:
with open("stories_qualtrics_advanced_txt.txt", "w", encoding='utf-8') as text_file:
    text_file.write(qualtrics_str)